# Mongabay Fire Tool Googel Cloud Function

[Getting started with Python for Google Cloud Functions](https://medium.com/@timhberry/getting-started-with-python-for-google-cloud-functions-646a8cddbb33).

**`main.py` file**

In [23]:
import os
import ee
import json
import requests
import numpy as np
import pandas as pd
import datetime as dt

#service_account = 'gee-tiles@skydipper-196010.iam.gserviceaccount.com'
#credentials = ee.ServiceAccountCredentials(service_account, 'privatekey.json')
#ee.Initialize(credentials)
ee.Initialize()

def serializer(df_pre, df_fire):

    return json.loads(pd.merge(df_fire, df_pre, how='left', on='date').to_json(orient='records'))

def get_geometry(iso, adm1=None):
    """
    Return a geometry as a GeoJSON from geostore.

    Parameters
    ----------
        iso : str
            Country iso code.
        adm1 : int or bool, optional
            Admin 1 code. Must be non-negative.

    Returns
    -------
        geometry : GeoJSON
            GeoJSON object describing the geometry.

    Examples
    --------
    >>> get_geometry('BRA', 22)
    {'crs': {},
    'features': [{'geometry': {'coordinates': [[[[-62.8922, -12.8601],
        [-62.8921, -12.8588],
        [-62.9457, -12.8571],
        ...]]],
        'type': 'MultiPolygon'},
    'properties': None,
    'type': 'Feature'}],
    'type': 'FeatureCollection'}
    """
    if adm1:
        if adm1 < 0:
            raise ValueError("Code number %s must be non-negative." % adm1)
        url = f'https://api.resourcewatch.org/v2/geostore/admin/{iso}/{adm1}'

    else:
        url = f'https://api.resourcewatch.org/v2/geostore/admin/{iso}'

    r = requests.get(url)
    geometry = r.json().get('data').get('attributes').get('geojson')

    return geometry

def get_dates(date_text=None):
    """
    Return relevant dates

    Parameters
    ----------
        date_text : str, optional
            String with the last date. Format should be YYYY-MM-DD.

    Returns
    -------
        dates : DatetimeIndex
            List of dates.
        start_date : Timestamp
            First date for moving window computation.
        end_date : Timestamp
            Last date for moving window computation.
        start_year_date : Timestamp
            First day of the 1 year range.    
    """
    if date_text:
        try:
            dt.datetime.strptime(date_text, '%Y-%m-%d')
        except ValueError:
            raise ValueError("Incorrect data format, should be YYYY-MM-DD")
        date = pd.to_datetime(date_text)

    else:
        date = pd.to_datetime('today').normalize()

    nDays_year =  len(pd.date_range(date.replace(month=1, day=1) , date.replace(month=12, day=31) ,freq='D'))
    start_year_date = date - dt.timedelta(days=nDays_year)
    start_date = date - dt.timedelta(days=nDays_year+61)
    end_date = date + dt.timedelta(days=61)
    dates = pd.date_range(start_date, end_date, freq='D').astype(str)
    dates = dates[(dates <= pd.to_datetime('today').normalize().strftime('%Y-%m-%d'))]

    return dates, start_date, date, end_date, start_year_date

def nestedMappedReducer(featureCol, imageCol):
    """
    Computes mean values for each geometry in a FeatureCollection and each image in an ImageCollection.
    To prevent "Computed value is too large" error we will map reduceRegion() over the FeatureCollection instead of using reduceRegions().

    Parameters
    ----------
        featureCol : ee.FeatureCollection
            FeatureCollection with the geometries that we want to intersect with.
        imageCol : ee.ImageCollection
            ImageCollection with a time series of images.

    Returns
    -------
        featureCol : ee.FeatureCollection
            FeatureCollection with the mean values for each geometry and date.  
    """
    def mapReducerOverImgCol(feature):
        def imgReducer(image):
            return ee.Feature(feature.geometry().centroid(100),
                image.reduceRegion(
                    geometry = feature.geometry(),
                    reducer = ee.Reducer.mean(),
                    tileScale = 10,
                    maxPixels = 1e+13,
                    bestEffort = True 
                )).set({'date': image.date().format("YYYY-MM-dd")}).copyProperties(feature)

        return imageCol.map(imgReducer)

    return featureCol.map(mapReducerOverImgCol).flatten()

def fire_tool(request):
    #request = request.get_json()
    request = request
    
    # Get geometry as GeoJSON from geostore
    geometry = get_geometry(request['iso'], request['adm1'])

    # Convert geometry to ee.Geometry
    aoi = ee.Geometry(geometry.get('features')[0].get('geometry'))

    # Get relevant dates
    dates, start_date, date, end_date, start_year_date = get_dates(request['date_text'])

    # Read ImageCollection
    dataset = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY') \
        .filter(ee.Filter.date(start_date.strftime('%Y-%m-%d'), (end_date + dt.timedelta(days=1)).strftime('%Y-%m-%d'))).filterBounds(aoi)
    chirps = dataset.select('precipitation')

    # Get mean precipitation values over time
    count = chirps.size()
    data = nestedMappedReducer(ee.FeatureCollection(geometry.get('features')), chirps).toList(count).getInfo()
    df_pre = pd.DataFrame(map(lambda x: x.get('properties'), data))

    # VIIRS fire alerts
    confidence = 'h' #'n', 'l'

    if request['adm1']:
        query =(f"SELECT alert__date, SUM(alert__count) AS alert__count \
                FROM data WHERE iso = \'{request['iso']}\' AND adm1::integer = {request['adm1']} AND confidence__cat = \'{confidence}\' AND alert__date >= \'{start_date}\' AND alert__date <= \'{end_date}\' \
                GROUP BY iso, adm1, alert__date, confidence__cat \
                ORDER BY alert__date"
        )
    else:
        query =(f"SELECT alert__date, SUM(alert__count) AS alert__count \
                FROM data WHERE iso = \'{request['iso']}\' AND confidence__cat = \'{confidence}\' AND alert__date >= \'{start_date}\' AND alert__date <= \'{end_date}\' \
                GROUP BY iso, alert__date, confidence__cat \
                ORDER BY alert__date"
        )

    url = f"https://data-api.globalforestwatch.org/dataset/gadm__viirs__adm2_daily_alerts/latest/query/json"

    sql = {"sql": query}
    r = requests.get(url, params=sql)

    data = r.json().get('data')
    if data: 
        df_fire = pd.DataFrame.from_dict(pd.json_normalize(data))
        # Fill missing dates with 0
        df_fire = df_fire.set_index('alert__date').reindex(dates, fill_value=0).reset_index().rename(columns={'index': 'alert__date'})
        df_fire.rename(columns = {'alert__date': 'date', 'alert__count': 'fire'}, inplace= True)
    else:
        df_fire = pd.DataFrame({"date": dates, 'fire': 0})

    # Moving averages
    # 1 week moving average
    df_pre['precipitation_w'] = df_pre[['date', 'precipitation']].rolling(window=7, center=True).mean()
    df_fire['fire_w'] = df_fire[['date', 'fire']].rolling(window=7, center=True).mean()
    # 2 month moving average
    df_pre['precipitation_2m'] = df_pre[['date', 'precipitation']].rolling(window=61, center=True).mean()
    df_fire['fire_2m'] = df_fire[['date', 'fire']].rolling(window=61, center=True).mean()
    # take current year days
    df_pre = df_pre[(df_pre['date'] >= start_year_date.strftime('%Y-%m-%d')) & (df_pre['date'] <= date.strftime('%Y-%m-%d'))]
    df_fire = df_fire[(df_fire['date'] >= start_year_date.strftime('%Y-%m-%d')) & (df_fire['date'] <= date.strftime('%Y-%m-%d'))]

    return json.dumps(serializer(df_pre, df_fire))


In [24]:
payload = {
    "iso": "BRA",
    "adm1": '',#22,
    "date_text": ''#'2020-10-10'
}

In [25]:
fire_tool(payload)

'[{"date": "2021-01-19", "fire": 19, "fire_w": 16.2857142857, "fire_2m": 24.5245901639, "precipitation": 10.1783731691, "precipitation_w": 7.9593605542, "precipitation_2m": 8.0486192929}, {"date": "2021-01-20", "fire": 16, "fire_w": 17.1428571429, "fire_2m": 24.1967213115, "precipitation": 7.7280316131, "precipitation_w": 7.5732291466, "precipitation_2m": 8.1366162802}, {"date": "2021-01-21", "fire": 8, "fire_w": 16.7142857143, "fire_2m": 24.131147541, "precipitation": 5.8904112751, "precipitation_w": 7.5731903175, "precipitation_2m": 8.226276396}, {"date": "2021-01-22", "fire": 12, "fire_w": 15.2857142857, "fire_2m": 23.8196721311, "precipitation": 6.2353554567, "precipitation_w": 7.5763050993, "precipitation_2m": 8.2700877828}, {"date": "2021-01-23", "fire": 25, "fire_w": 13.7142857143, "fire_2m": 24.4426229508, "precipitation": 7.0891128928, "precipitation_w": 7.2681786668, "precipitation_2m": 8.2513188591}, {"date": "2021-01-24", "fire": 12, "fire_w": 12.0, "fire_2m": 24.1803278689

### Googel Cloud Function
**Deploy function**

In [ ]:
!cd ../cloud_function/fire_tool

In [ ]:
!gcloud functions deploy fire_tool --runtime python38 --trigger-http --timeout 540 --allow-unauthenticated

**Request function**

In [11]:
from pprint import pprint
import json
import requests

In [30]:
payload = {
    "iso": "BRA",
    "adm1": None, #22,
    "date_text": '2021-10-10'
}

In [31]:
url = f'https://us-central1-fire-water-chart.cloudfunctions.net/fire_water_chart'

headers = {'Content-Type': 'application/json'}

r = requests.post(url, data=json.dumps(payload), headers=headers)
pprint(r.json())

[{'date': '2020-10-10',
  'fire': 1471,
  'fire_2m': 1037.9344262295,
  'fire_w': 1351.7142857143,
  'precipitation': 3.4790379842,
  'precipitation_2m': 3.2704856626,
  'precipitation_w': 2.618913792},
 {'date': '2020-10-11',
  'fire': 360,
  'fire_2m': 1013.0655737705,
  'fire_w': 1187.4285714286,
  'precipitation': 4.4609568604,
  'precipitation_2m': 3.3231709162,
  'precipitation_w': 2.7997660046},
 {'date': '2020-10-12',
  'fire': 327,
  'fire_2m': 966.7868852459,
  'fire_w': 891.5714285714,
  'precipitation': 1.7485759726,
  'precipitation_2m': 3.3853917886,
  'precipitation_w': 3.1723082159},
 {'date': '2020-10-13',
  'fire': 558,
  'fire_2m': 907.868852459,
  'fire_w': 662.8571428571,
  'precipitation': 2.0621667602,
  'precipitation_2m': 3.44790053,
  'precipitation_w': 3.0817650254},
 {'date': '2020-10-14',
  'fire': 684,
  'fire_2m': 847.3442622951,
  'fire_w': 551.0,
  'precipitation': 3.3466497836,
  'precipitation_2m': 3.5283329712,
  'precipitation_w': 2.8082798763},
 {'

In [5]:
!curl -X POST https://us-central1-skydipper-196010.cloudfunctions.net/fire_tool"https://us-central1-fire-water-chart.cloudfunctions.net/fire_water_chart" -H "Content-Type:application/json" --data '{"iso": "BRA", "adm1": 22, "date_text": "2021-10-10"}'

[{"date": "2020-10-10", "fire": 356, "fire_w": 93.5714285714, "fire_2m": 34.5245901639, "precipitation": 0.2018636235, "precipitation_w": 2.2670981327, "precipitation_2m": 5.1149473581}, {"date": "2020-10-11", "fire": 16, "fire_w": 92.5714285714, "fire_2m": 34.0819672131, "precipitation": 5.2612908266, "precipitation_w": 3.5613245354, "precipitation_2m": 5.1539803312}, {"date": "2020-10-12", "fire": 9, "fire_w": 86.4285714286, "fire_2m": 33.9344262295, "precipitation": 3.1359100564, "precipitation_w": 4.3597407212, "precipitation_2m": 5.1847560883}, {"date": "2020-10-13", "fire": 9, "fire_w": 66.4285714286, "fire_2m": 33.0, "precipitation": 5.1519799746, "precipitation_w": 4.9503342209, "precipitation_2m": 5.2079660651}, {"date": "2020-10-14", "fire": 32, "fire_w": 18.1428571429, "fire_2m": 31.7049180328, "precipitation": 9.4813654909, "precipitation_w": 5.1191926729, "precipitation_2m": 5.2299289004}, {"date": "2020-10-15", "fire": 2, "fire_w": 18.5714285714, "fire_2m": 30.6885245902,

**Handling CORS requests**

Cross-Origin Resource Sharing (CORS) is a way to let applications running on one domain access content from another domain, for example, letting `yourdomain.com` make requests to `region-project.cloudfunctions.net/yourfunction`.

If CORS isn't set up properly, you're likely to get errors that look like this:


```js
XMLHttpRequest cannot load https://region-project.cloudfunctions.net/function.
No 'Access-Control-Allow-Origin' header is present on the requested resource.
Origin 'http://yourdomain.com' is therefore not allowed access.
```

CORS consists of two requests, a pre-flight request followed by the main request. 

To resolve this issue, [Google’s documentation](https://cloud.google.com/functions/docs/writing/http#handling_cors_requests) recommends setting the proper headers in the Cloud Funtion itself by adding the following code:

```python
# Set CORS headers for the preflight request
if request.method == 'OPTIONS':
    # Allows GET requests from any origin with the Content-Type
    # header and caches preflight response for an 3600s
    headers = {
        'Access-Control-Allow-Origin': '*',
        'Access-Control-Allow-Methods': 'PUT',
        'Access-Control-Allow-Headers': 'Content-Type',
        'Access-Control-Max-Age': '3600'
    }

    return ('', 204, headers)

# Set CORS headers for the main request
headers = {
    'Access-Control-Allow-Origin': '*'
}
```

The updated version of the Google Cloud Function can be found below.
 
As you can see, there is an additional `if` block that sets the correct headers for a pre-flight request with the `OPTIONS` method. 

After the correct headers have been set, the main request is made with the output of the function, a 200 response code, and the CORS headers – `(json.dumps(serializer(df_pre, df_fire)), 200, headers)`.

```python
def fire_water_chart(request):

    # Set CORS headers for the preflight request
    if request.method == 'OPTIONS':
        # Allows GET requests from any origin with the Content-Type
        # header and caches preflight response for an 3600s
        headers = {
            'Access-Control-Allow-Origin': '*',
            'Access-Control-Allow-Methods': 'POST',
            'Access-Control-Allow-Headers': 'Content-Type',
            'Access-Control-Max-Age': '3600'
        }

        return ('', 204, headers)

    # Set CORS headers for the main request
    headers = {
        'Access-Control-Allow-Origin': '*'
    }

    request_json = request.get_json()
    
    # Get geometry as GeoJSON from geostore
    geometry = get_geometry(request_json['iso'], request_json['adm1'])

    # Convert geometry to ee.Geometry
    aoi = ee.Geometry(geometry.get('features')[0].get('geometry'))

    # Get relevant dates
    dates, start_date, end_date, start_year_date = get_dates(request_json['date_text'])

    # Read ImageCollection
    dataset = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY') \
        .filter(ee.Filter.date(start_date.strftime('%Y-%m-%d'), (end_date + dt.timedelta(days=1)).strftime('%Y-%m-%d'))).filterBounds(aoi)
    chirps = dataset.select('precipitation')

    # Get mean precipitation values over time
    count = chirps.size()
    data = nestedMappedReducer(ee.FeatureCollection(geometry.get('features')), chirps).toList(count).getInfo()
    df_pre = pd.DataFrame(map(lambda x: x.get('properties'), data))

    # VIIRS fire alerts
    confidence = 'h' #'n', 'l'

    if request_json['adm1']:
        query =(f"SELECT alert__date, SUM(alert__count) AS alert__count \
                FROM data WHERE iso = \'{request_json['iso']}\' AND adm1::integer = {request_json['adm1']} AND confidence__cat = \'{confidence}\' AND alert__date >= \'{start_date}\' AND alert__date <= \'{end_date}\' \
                GROUP BY iso, adm1, alert__date, confidence__cat \
                ORDER BY alert__date"
        )
    else:
        query =(f"SELECT alert__date, SUM(alert__count) AS alert__count \
                FROM data WHERE iso = \'{request_json['iso']}\' AND confidence__cat = \'{confidence}\' AND alert__date >= \'{start_date}\' AND alert__date <= \'{end_date}\' \
                GROUP BY iso, alert__date, confidence__cat \
                ORDER BY alert__date"
        )

    url = f"https://data-api.globalforestwatch.org/dataset/gadm__viirs__adm2_daily_alerts/latest/query/json"

    sql = {"sql": query}
    r = requests.get(url, params=sql)

    data = r.json().get('data')
    if data: 
        df_fire = pd.DataFrame.from_dict(pd.json_normalize(data))
        # Fill missing dates with 0
        df_fire = df_fire.set_index('alert__date').reindex(dates, fill_value=0).reset_index().rename(columns={'index': 'alert__date'})
        df_fire.rename(columns = {'alert__date': 'date', 'alert__count': 'fire'}, inplace= True)
    else:
        df_fire = pd.DataFrame({"date": dates, 'fire': 0})

    # Moving averages
    # 1 week moving average
    df_pre['precipitation_w'] = df_pre[['date', 'precipitation']].rolling(window=7, center=True).mean()
    df_fire['fire_w'] = df_fire[['date', 'fire']].rolling(window=7, center=True).mean()
    # 2 month moving average
    df_pre['precipitation_2m'] = df_pre[['date', 'precipitation']].rolling(window=61, center=True).mean()
    df_fire['fire_2m'] = df_fire[['date', 'fire']].rolling(window=61, center=True).mean()
    # take current year days
    df_pre = df_pre[(df_pre['date'] >= start_year_date.strftime('%Y-%m-%d')) & (df_pre['date'] <= end_date.strftime('%Y-%m-%d'))]
    df_fire = df_fire[(df_fire['date'] >= start_year_date.strftime('%Y-%m-%d')) & (df_fire['date'] <= end_date.strftime('%Y-%m-%d'))]

    return (json.dumps(serializer(df_pre, df_fire)), 200, headers)
```